# Kőnig's theorem

## Minimal vertex cover in a bipartite graph

A *vertex cover* in a graph is a set of vertices that includes at least one endpoint of every edge, and a vertex cover is minimum if no other vertex cover has fewer vertices. A *matching* in a graph is a set of edges no two of which share an endpoint, and a matching is maximum if no other matching has more edges.

It is obvious from the definition that any vertex cover must be with at least the same size as any matching (since for every edge in the matching, at least one vertex is needed in the cover). In particular, the minimum vertex cover set is at least as large as the maximum matching set. Meanwhile, all the vertices in a maximal matching is obviously a vertex cover, so the size of a minimal vertex cover is as most double the size of a maximal matching.

For general graphs, to find a maximal matching is not hard. However, unfortunately, to find a minimal vertex cover is NP-complete. Hence, in this page, we will only discuss the cases of bipartite graphs where, these two problems are equivalent to each other.

## Kőnig's theorem

Kőnig's theorem, proved by Dénes Kőnig (Dénes Kőnig (September 21, 1884 – October 19, 1944) was a Hungarian mathematician of Hungarian Jewish heritage who worked in and wrote the first textbook on the field of graph theory.) in 1931, describes the equivalence between the maximum matching problem and the minimum vertex cover problem in bipartite graphs. It was discovered independently, also in 1931, by Jenő Egerváry in the more general case of weighted graphs.

**Kőnig's theorem.**
In any bipartite graph, the number of edges in a maximum matching equals the number of vertices in a minimum vertex cover.

**Proof 1.** (Using Hall's Marriage Theorem)
We shall show that we can find a maximal matching whose number of edges is equal to the number of vertices in a minimal vertex cover. Given a bipartite graph $G = (X\sqcup Y,E)$ and a minimal vertex cover $C = S\sqcup T$ with $S \subseteq X, T \subseteq Y$. For the induced subgraphs $G_1 = G(S\sqcup (Y\setminus T))$ and $G_2 = G(T\sqcup(X\setminus S))$, we can check the Hall's marriage condition holds on both cases, which will give us a perfect matching $M_1$ in $G_1$ and $M_2$ in $G_2$. $M = M_1 \sqcup M_2$ is a maximal matching we are looking for.
<p style="text-align:right;">&#9632; </p>

The proof above is mathematically easy, but not constructive, which is hard to be used to give any minimal vertex cover.

**Proof 2.** (Constructive proof)
We shall show that we can find a minimal vertex cover whose number of vertices is equal to the number of edges in a maximal matching. Given a bipartite graph $G = (X\sqcup Y,E)$ and a maximal matching $M$ such that $M \bigcap X = S_0$ and $M \bigcap Y = T_0$. Using $M$, we can assign directions for each edge in $G$ by
* $e = \{x,y\}$ goes from $x$ to $y$ if $e \notin M$
* $e = \{x,y\}$ goes from $y$ to $x$ if $e \in M$

Consider the connect component $C_x$ of $x \in X \setminus S_0$ and let $C_x \bigcap X = S_x$ and $C_x \bigcap Y = T_x$, $T_x \subseteq T_0$ by the assumption that $M$ is maximal. Let
$$
    C = \Big(S_0\setminus \big(\bigcup_{x\in X\setminus S_0} S_x\big)\Big) \bigsqcup \Big(\bigcup_{x\in X\setminus S_0} T_x\Big)
$$

Then for any $e = (x,y)\in E$, either $x\in S_0$ or $y \in T_0$.
- If $x \in S_0$, then either $x \in C$, or $y \in C$
- If $x \notin S_0$, then $y\in T_x \subseteq C$

Hence, $C$ is a vertex cover.
<p style="text-align:right;">&#9632; </p>

**Remark.** Hopcroft–Karp–Karzanov algorithm then provides a very effective method to find a minimal vertex cover in a bipartite graph.

Input a 2D list ```data``` in the following widget, you will see how this algorithm works.

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["animation.html"] = "jshtml"
import matplotlib.animation
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import random
import matplotlib.patches as patch
from IPython.display import display,clear_output
from celluloid import Camera
from collections import defaultdict


output_widget2 = widgets.Output()

class BipartiteGraph(object):
    def __init__(self, data):
        m = len(data)
        n = max(max([-1] + u) for u in data) + 1
        self.__NIL = m
        self.__INF = float('INF')
        self.Pair_U = [m] * m
        self.Pair_V = [m] * n
        self.Dist = [self.__INF] * (m + 1)
        self.edges = data
        self.m,self.n = m,n
        self.size = max(m,n)
        self.path = defaultdict(list)
        m = self.size
        self.fig,self.ax = plt.subplots()
        self.camera = Camera(self.fig)
        
        self.ax.set_axis_off()

        self.ax.set_xlim(-1.3,1.7)
        self.ax.set_ylim(-.5,m + 1.5)
        self.ax.set_aspect('equal')
        self.hint = {'x': -1.3, 'y': m + 0.5, 'text':'The bipartite graph is drawn!'}
        self.colors = {
            0:'#FEF9E7', 
            1:'#F9E79F', 
            2:'#F4D03F', 
            3:'#F1C40F', 
            4:'#D4AC0D', 
            5:'#B7950B', 
            6:'#9A7D0A', 
            self.__INF:'#7D6608'}
        self.draw()
        
        
    def __str__(self):
        return str([self.__NIL,
        self.__INF ,
        self.Pair_U,
        self.Pair_V,
        self.Dist,
        self.edges,
        self.path])
    
          
    def BFS(self):
        Q = []
        for u in range(self.m):
            if self.Pair_U[u] == self.__NIL:
                self.Dist[u] = 0 
                Q.append(u)
            else:
                self.Dist[u] = self.__INF
            self.Dist[self.__NIL] = self.__INF
        while Q:
            u = Q.pop(0)
            if self.Dist[u] < self.Dist[self.__NIL]:
                for v in self.edges[u]:
                    if self.Dist[self.Pair_V[v]] == self.__INF:
                        self.Dist[self.Pair_V[v]] = self.Dist[u] + 1
                        Q.append(self.Pair_V[v])
        if self.Dist[self.__NIL] != self.__INF:
            return True
        else:
            return False

    def DFS(self, u):
        if u !=self.__NIL:
            for v in self.edges[u]:
                if self.Dist[self.Pair_V[v]] ==self.Dist[u] + 1:
                    if self.DFS(self.Pair_V[v]):
                        self.Pair_V[v] = u
                        self.Pair_U[u] = v
                        return True
            self.Dist[u] =self.__INF
            return False
        return True
    
    def draw(self):
        self.ax.text(
            -1.3, self.size+1,
            'Hopcroft–Karp–Karzanov algorithm', 
            fontsize = 11,
            weight='bold')

        x,y,text = self.hint['x'],self.hint['y'],self.hint['text']
        self.ax.text(x,y,text,fontsize = 10,va = 'top')
        
        for g in range(self.m):
            for b in self.edges[g]:
                x0,y0 = -0.8,g + 0.1
                line, = self.ax.plot(np.array([x0,1.2]), np.array([y0,b+0.1]),color = '#808080',zorder = 0)
            if self.Pair_U[g] != self.__NIL:
                b = self.Pair_U[g]
                x0,y0 = -0.8,g + 0.1
                line, = self.ax.plot(np.array([x0,1.2]), np.array([y0,b+0.1]),color = '#FF0000',zorder = 0)
                
        u_boxes = {}
        for g in range(self.m):
            col = self.colors[3]
            box = patch.FancyBboxPatch(
                (-1, g), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = col,
                zorder = 5
            )
            self.ax.add_artist(box)
            text = self.ax.text(
                -0.8, 0.1+g,
                f'U[{g}]', 
                color='k', 
                style = 'italic',
                weight='bold', 
                fontsize=12, 
                ha='center', 
                va='center',
                zorder = 10)
            u_boxes[g] = {'box':box,'text':text}
        
        v_boxes = {}
        c = 0
        for b in range(self.n):
            box = patch.FancyBboxPatch(
                (1, b), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = '#0066CC',
                zorder = 5
            )
            self.ax.add_artist(box)
            text = self.ax.text(
                1.2, 0.1+b,
                f'V[{b}]', 
                color='k', 
                style = 'italic',
                weight='bold', 
                fontsize=12, 
                ha='center', 
                va='center',
                zorder = 10
            )
            v_boxes[b] = {'box':box,'text':text,'x':1.2,'y':b+0.1}
        self.camera.snap()
        
    def draw2(self):
        self.ax.text(
            -1.3, self.size+1,
            'Hopcroft–Karp–Karzanov algorithm', 
            fontsize = 11,
            weight='bold')

        x,y,text = self.hint['x'],self.hint['y'],self.hint['text']
        self.ax.text(x,y,text,fontsize = 10,va = 'top')
        
        for g in range(self.m):
            for b in self.edges[g]:
                x0,y0 = -0.8,g + 0.1
                line, = self.ax.plot(np.array([x0,1.2]), np.array([y0,b+0.1]),color = '#808080',zorder = 0)
            if self.Pair_U[g] != self.__NIL:
                b = self.Pair_U[g]
                x0,y0 = -0.8,g + 0.1
                line, = self.ax.plot(np.array([x0,1.2]), np.array([y0,b+0.1]),color = '#FF0000',zorder = 0)
            
                
        u_boxes = {}
        for g in range(self.m):
            if self.Dist[g] == self.__INF: 
                col = self.colors[3]
            else:
                col = 'w'
            box = patch.FancyBboxPatch(
                (-1, g), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = col,
                zorder = 5
            )
            self.ax.add_artist(box)
            text = self.ax.text(
                -0.8, 0.1+g,
                f'U[{g}]', 
                color='k', 
                style = 'italic',
                weight='bold', 
                fontsize=12, 
                ha='center', 
                va='center',
                zorder = 10)
            u_boxes[g] = {'box':box,'text':text}
        
        v_boxes = {}
        c = 0
        for b in range(self.n):
            if self.Dist[self.Pair_V[b]] != self.__INF:
                col = '#0066CC'
            else:
                col = 'w'
            box = patch.FancyBboxPatch(
                (1, b), 0.4, 0.2,
                boxstyle=patch.BoxStyle("Round", pad=0.2),
                color = col,
                zorder = 5
            )
            self.ax.add_artist(box)
            text = self.ax.text(
                1.2, 0.1+b,
                f'V[{b}]', 
                color='k', 
                style = 'italic',
                weight='bold', 
                fontsize=12, 
                ha='center', 
                va='center',
                zorder = 10
            )
            v_boxes[b] = {'box':box,'text':text,'x':1.2,'y':b+0.1}
        self.camera.snap()

    def HKK(G):
        matching = 0
        while G.BFS():
            for u in range(G.m):
                
                if G.Pair_U[u] == G.__NIL:
                    if G.DFS(u):
                        matching = matching + 1

        G.hint['text'] = f'The Hopcroft–Karp–Karzanov algorithm stops.\nThe maximal matching has {matching} pairs.'
        G.draw()
        G.BFS()
        G.hint['text'] = f'A minimal vertex cover is selected\nwith {matching} vertices.'
        G.draw2()
        ani = G.camera.animate(interval = 1000)
        with output_widget2:
            clear_output()
            display(ani)

input_text2 = widgets.Text(
    value='',
    placeholder='Input a 2D list like [[0,],[2,],[1,3,]]',
    description='Input array',
    disabled=False,  
    continuous_update= False
)

display(input_text2)

def HKK_widget(change):
    a = change['new']
    with output_widget2:
        
        clear_output()
        
        # check if the input is a 2D list ###
        if not a:
            print('Input is empty, try to input a 2D list')
            return
        try:
            data = eval(a)
        except:
            print("Illegal inputs")
            return
        try:
            if type(data) != list:
                print("Input should be like [[0,],[2,],[1,3,]]")
                return
            if not data:
                print('Input is empty, try to input a 2D list')
                return
            for girl in data:
                if type(girl) != list:
                
                    print("Input should be like [[0,],[2,],[1,3,]]")
                    for boy in girl:
                        if type(boy) != int:
                            raise Exception("Illegal inputs")
                    return
        except:
            print("Illegal inputs")
            return
    G = BipartiteGraph(data) 
    G.HKK()
    
input_text2.observe(HKK_widget, names = 'value')
display(output_widget2)

Text(value='', continuous_update=False, description='Input array', placeholder='Input a 2D list like [[0,],[2,…

Output()

## Bipartite graphs as partially ordered sets

**Rethink.** A bipartite graph is a special finite partially ordered set, such that any chain in it is not longer than $2$. To be explicit, given $G = (X\sqcup Y, E)$, $P = X \sqcup Y$ carries a partial order $\leqslant_E$, $\forall x,y \in P, x \leqslant_E y \iff x \in X ,y \in Y$ and $(x,y) \in E$. Then, Kőnig's theorem for bipartite graphs is translated into Dilworth's theorem for finite posets,

**Dilworth's theorem.**
In any finite partially ordered set, the largest antichain has the same size as the smallest chain decomposition.

**Proof 3.** (Using Dilworth's theorem)
- Matchings $\{M\}$ in $G$ are one-to-one corresponding to chain decompositions $\{\mathcal{C}\}$ with $|X| + |Y| - |M| = |\mathcal{C}|$
- Vertex covers $\{S\sqcup T\}$ in $G$ are one-to-one corresponding to antichains $\{P\setminus (S \sqcup T)\}$ with $|P| - |S\sqcup T| = |P \setminus (S \sqcup T)|$
The result follows.
<p style="text-align:right;">&#9632; </p>

**Remark.** Kőnig's theorem and Dilworth's theorem are in fact equivalent.